In [ ]:
import logging
import sys
import os
%cd ..
from bds.meel import approx_mc2, approx_mc2_core
from bds.rule import Rule
from bds.utils import bin_random , bin_array, randints, solutions_to_dict

#from bds.bb import BranchAndBoundNaive, BranchAndBoundV1, BranchAndBoundV0, BranchAndBoundV2
from logzero import logger
import gmpy2 as gmp
from gmpy2 import mpz , mpfr
from bds.sat.min_freq import construct_min_freq_program
from bds.sat.solver import construct_solver
from bds.gf2 import GF
from bds.rule import Rule
from bds.utils import mpz_set_bits
from tests.utils import assert_dict_allclose, assert_close_mpfr

#from bds.bounds_utils import find_equivalence_classes
import numpy as np 

from bds.cbb_v2 import * 

logger.setLevel(logging.DEBUG)

num_pts = 500

show_progres = True
rand_seed = 1234

In [ ]:
from time import time

#num_rules_list = reversed([25, 50, 75, 100, 200])
ub = 0.7
lmbd = 0.1
res_rows = []
all_times_naive = []
all_times_V1 = [] 
all_times_V0 = [] 

for num_rules in [5]:
    
    rules = [Rule.random(i+1, num_pts) for i in range(num_rules)]
    y = bin_random(num_pts)
    
    start_time = time() 
    bbNaive = BranchAndBoundNaive(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbNaive.run(return_objective=True))
    all_times_naive.append(time() - start_time) 
    actualNaive = solutions_to_dict(feasible_solutions)
    
    
    start_time = time() 
    bbV0 = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV0.run(return_objective=True))
    all_times_V0.append(time() - start_time)
    actual_V0 = solutions_to_dict(feasible_solutions)

    
    
    start_time = time() 
    bbV1 = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV1.run(return_objective=True))
    all_times_V1.append(time() - start_time)
    actual_V1 = solutions_to_dict(feasible_solutions)
    

In [ ]:
os.chdir("/u/50/ciaperm1/unix/Desktop/sampling-rashomon-decision-set-code")

In [ ]:
import matplotlib.pyplot as plt 

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

# plot 
fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
plt.plot([10, 25, 40], all_times_naive, label="HLB", marker = "o", linewidth=1.5 , color = colors[0], markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot([10, 25, 40], all_times_V0, label="RSSB", marker = "^",  linewidth=1.5 ,  color = colors[1],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot([10, 25, 40], all_times_V1, label="RSSB + EPLB", marker = "s", linewidth=1.5 ,  color = colors[2],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.yscale("log") 
plt.title("RSSB VS HLB VS RSSB + EPLB") 
plt.xlabel("# Rules")
plt.legend()
plt.grid()
plt.tight_layout() 
plt.savefig("plots/log_first_comparison_bounds" + ".pdf")
plt.show()


fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
plt.plot([10, 25, 40], all_times_naive, label="HLB", marker = "o", linewidth=1.5 , color = colors[0],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot([10, 25, 40], all_times_V0, label="RSSB", marker = "^", linewidth=1.5 , color = colors[1],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot([10, 25, 40], all_times_V1, label="RSSB + EPLB",  marker = "s", linewidth=1.5 ,  color = colors[2],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.title("RSSB VS HLB VS RSSB + EPLB") 
plt.xlabel("# Rules")
plt.legend()
plt.grid()
plt.tight_layout() 
plt.savefig("plots/first_comparison_bounds" + ".pdf" )
plt.show()

#### Check same results 

In [ ]:
from time import time

#num_rules_list = reversed([25, 50, 75, 100, 200])
ub = 0.7
lmbd = 0.1
res_rows = []
all_times_naive = []
all_times_V1 = [] 
all_times_V0 = [] 

for num_rules in [10]:
    
    rules = [Rule.random(i+1, num_pts) for i in range(num_rules)]
    y = bin_random(num_pts)
    
    start_time = time() 
    bbNaive = BranchAndBoundNaive(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbNaive.run(return_objective=True))
    all_times_naive.append(time() - start_time) 
    actualNaive = solutions_to_dict(feasible_solutions)
    
    
    start_time = time() 
    bbV0 = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV0.run(return_objective=True))
    all_times_V0.append(time() - start_time)
    actual_V0 = solutions_to_dict(feasible_solutions)
    
    assert_dict_allclose(actualNaive, actual_V0)
    
    start_time = time() 
    bbV1 = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV1.run(return_objective=True))
    all_times_V1.append(time() - start_time)
    actual_V1 = solutions_to_dict(feasible_solutions)
    
    assert_dict_allclose(actualNaive, actual_V1)

In [ ]:
actualNaive


### try real world data 

In [1]:
from sklearn import datasets 
import logging
import sys
import os
%cd ..
from bds.meel import approx_mc2, approx_mc2_core
from bds.rule import Rule
from bds.utils import bin_random , bin_array, randints, solutions_to_dict

#from bds.bb import BranchAndBoundNaive, BranchAndBoundV1, BranchAndBoundV0, BranchAndBoundV2
from logzero import logger
import gmpy2 as gmp
from gmpy2 import mpz , mpfr
from bds.sat.min_freq import construct_min_freq_program
from bds.sat.solver import construct_solver
from bds.gf2 import GF
from bds.rule import Rule
from bds.utils import mpz_set_bits
from tests.utils import assert_dict_allclose, assert_close_mpfr

#from bds.bounds_utils import find_equivalence_classes
import numpy as np 

#from bds.cbb_v2 import * 

logger.setLevel(logging.DEBUG)

num_pts = 500

show_progres = True
rand_seed = 1234

/u/50/ciaperm1/unix/sampling-rashomon-decision-set-code


In [2]:
import pandas as pd

In [3]:
# Load the CSV file using Pandas
data = pd.read_csv('data/compas_test-binary.csv')

# Print the data
data.head()

,sex:Male,age:18-20,age:21-22,age:23-25,age:26-45,age:>45,juvenile-felonies:>0,juvenile-misdemeanors:>0,juvenile-crimes:>0,priors:2-3,priors:=0,priors:=1,priors:>3,current-charge-degree:Misdemeanor,recidivate-within-two-years:1
0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1
2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1
3,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1
4,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0


In [4]:
data.shape

(721, 15)

In [5]:
X = data.to_numpy()

In [6]:
X.shape

(721, 15)

In [7]:
# read labels 
f = open("data/compas_test.label", "r")
labels = []
for row in f.readlines(): 
    labels.append(list(map(int, row.split(" ")[1:])))

In [8]:
y = np.array(labels[1], dtype = bool) 

In [9]:
assert len(y) == X.shape[0]

In [10]:
# now we need to extract rules 
X_bag = [set([j for j, x in enumerate(row) if x]) for row in X]

In [11]:
from bds.fpgrowth import preprocess_transaction_list, build_fptree, fpgrowth_on_tree

In [12]:
lmbd = 0.1
min_support = lmbd * X.shape[0] / 2

In [13]:
ordered_input_data = preprocess_transaction_list(
    X_bag, min_support
)
tree = build_fptree(ordered_input_data)
frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))

In [14]:
len(frequent_itemsets)

83

In [15]:
from bds.rule import Rule
from time import time

def compute_truthtable(X,itemset): 
    ''' poorly optimized function '''
    support_list = [] 
    for i in range(X.shape[0]): 
        if sum([X[i][j] for j in itemset]) == len(itemset): 
            support_list.append(i)
                
    truthtable = mpz_set_bits(mpz(), support_list)
    
    return truthtable
    

In [16]:
# Now create rules 
rules = [] 
for i,itemset in enumerate(frequent_itemsets):
    tt = compute_truthtable(X,itemset)
    this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
    # we need the truthtable for this itemset 
    rules.append(this_Rule) 
    

In [17]:
import pickle 
pickle.dump(rules, open("rules_compas_" + str(len(rules)), "wb" ))

In [18]:
from bds.bbSampleTree import BranchAndBoundNaive
import time 
import random

In [21]:
lmbd = 0.1
ub = 0.7
bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=4, k=150) 
ll = list(bb.runST())
ll

[D 230628 15:01:56 bbSampleTree:60] calling BranchAndBoundNaive with ub=0.7, lmbd=0.1


16
23
33
48
62
72
79
83
97
108
123
138
152
167
182
196
210
152
154
161
168
175
182
182
here!
here!
here!
here!
182
185
188
here!
here!
here!
here!
188
195
198
200
202
207
222
225
229
here!
here!
here!
here!
229
here!
here!
here!
here!
229
236
here!
here!
here!
here!
236
here!
here!
here!
here!
236
243
250
253
260
267
278
289
291
here!
here!
here!
here!
291
294
305
308
here!
here!
here!
here!
308
here!
here!
here!
here!
308
311
here!
here!
here!
here!
311
318
332
339
here!
here!
here!
here!
339
349
here!
here!
here!
here!
349
here!
here!
here!
here!
349
357
362
366
369
376
376
378
380
387
389
392
399
here!
here!
here!
here!
399
here!
here!
here!
here!
399
401
405
here!
here!
here!
here!
405
412
here!
here!
here!
here!
412
423
430
here!
here!
here!
here!
430
433
440
448
462
464
471
474
476
here!
here!
here!
here!
476
here!
here!
here!
here!
476
here!
here!
here!
here!
476
483
490
497
501
508
510
515
517
523
526
529
here!
here!
here!
here!
529
531
535
here!
here!
here!
here!
535
539
here!

[({0, 7, 12, 22, 40, 42, 59},
  (Node(rule_id=59, lower_bound=0.6651872399445214, objective=0.6651872399445214),
   mpz(3189954091506676374710603162167398284425188360560570998521111661812681522382279112925177246442834424129123060568669998475381806838824520325495639745426830807150714131380476411457923405285225353588930658732092297780305)),
  0.6651872399445214),
 ({0, 7, 12, 22, 61, 68},
  (Node(rule_id=68, lower_bound=0.5651872399445215, objective=0.6345353675450762),
   mpz(3879410624555109384511585032873008397218316877578894463018178288105116236587770158635864447754704498636789718186814367755029880709087833826761326227800505994923803991995857751424419222446358038867121759629518476615281)),
  0.6345353675450762),
 ({0, 7, 12, 22, 65},
  (Node(rule_id=65, lower_bound=0.4693481276005548, objective=0.5373092926490984),
   mpz(38794106245551093843620829787158520749031717054213646253682085955447248705122175210137851835906988897197558045842424643911804691387473561493024515393058569598170944

In [ ]:
len(rules)

In [ ]:
bb.reset_tree()

In [ ]:
bb.tree.root

In [ ]:
pseudosolutions = [bb.tree.root] # empty -- 

In [ ]:
sampled_items = random.sample(pseudosolutions, 1)

In [ ]:
sampled_items

In [ ]:
not_captured_root = bb._not_captured_by_default_rule()



In [ ]:
pseudosolutions = [(bb.tree._root, not_captured_root)] # empty -
pseudosolutions     

In [ ]:
out = list(bb._loop( *pseudosolutions[0] , 1 , return_objective=False))

In [ ]:
set(out)

In [ ]:
input_set = {'0', '4-57'}

output_list = [int(part) for item in input_set for part in item.split('-')]
print(output_list)

In [ ]:
import gmpy2 as gmp

In [ ]:
a1 = np.array([0,0,1,1,0,1]) 
a2 = np.array([0,1,0,1,0,1]) 
def mpz_set_bits(n: mpz, bits: np.ndarray) -> mpz:
    """return a copy of n and set `bits` to 1 in `n`"""
    for i in bits:
        n = gmp.bit_set(n, int(i))
    return n


In [ ]:
a1_mpz = mpz_set_bits( mpz(), a1.nonzero()[0])
a2_mpz = mpz_set_bits( mpz(), a2.nonzero()[0])

In [ ]:
a3_mpz = mpz_set_bits( mpz(), a3.nonzero()[0]) 

In [ ]:
a3_mpz

In [ ]:
logicalAnda = a1_mpz | a2_mpz

In [ ]:
logicalAnda

In [ ]:
def mpz2bag(n: mpz):
    """given a mpz() this function returns the indices of non-zero entries"""
    i = 0
    bag = set()
    thisBit = gmp.bit_scan1(n, i)
    while thisBit is not None:
        bag.add(thisBit)
        i += 1
        thisBit = gmp.bit_scan1(n, i)

    return bag

mpz2bag(a1_mpz)

In [ ]:
a1, a2

In [ ]:
mpz2bag(logicalAnda)

In [ ]:
def compute_union(vectors):
    result = vectors[0]  # Initialize result with the first vector
    for vector in vectors[1:]:
        result = result | vector   # Perform bitwise OR operation
    return result

a3 = np.array([1,0,0,0,0,1])

In [ ]:
out = compute_union([a1, a2, a3]) 
out

In [ ]:
out = compute_union([a1_mpz, a2_mpz, a3_mpz]) 
out

In [ ]:
mpz2bag(out)

In [ ]:
ordered_input_data = preprocess_transaction_list(
    X_bag, min_support
)
tree = build_fptree(ordered_input_data)

all_times_naive = [] 
all_times = [] 
all_times_approx_mc2= []

eps = 0.75
delta = 0.75

for min_support in [30, 40 , 50, 60]:  
    
    
    #print("min support " + str(min_support)) 
    #print("tree " + str(tree))

    frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))
    # Now create rules 
    rules = [] 
    for i,itemset in enumerate(frequent_itemsets):
        tt = compute_truthtable(X,itemset)
        this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
        # we need the truthtable for this itemset 
        rules.append(this_Rule) 
    
    print(len(rules))

In [ ]:
ordered_input_data = preprocess_transaction_list(
    X_bag, min_support
)
tree = build_fptree(ordered_input_data)

all_times_naive = [] 
all_times = [] 
all_times_approx_mc2= []

eps = 0.75
delta = 0.75

for min_support in [30, 40 , 50, 60]:  
    
    
    #print("min support " + str(min_support)) 
    #print("tree " + str(tree))

    frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))
    # Now create rules 
    rules = [] 
    for i,itemset in enumerate(frequent_itemsets):
        tt = compute_truthtable(X,itemset)
        this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
        # we need the truthtable for this itemset 
        rules.append(this_Rule) 
    
    #    
    #
    
    print(len(rules))
    
    pickle.dump(rules, open("rules_compas_" + str(len(rules)), "wb" ))

    lmbd = 0.1
    ub = 0.4 
    cbb = ConstrainedBranchAndBoundNaive(rules, ub, y, lmbd)
    cbb_v2 = ConstrainedBranchAndBound(rules, ub, y, lmbd)
    
    #
    #
    
    rand_seed = 12
    num_constraints = 5
    num_rules = len(rules)
    
    from bds.random_hash import generate_h_and_alpha
    A, t = generate_h_and_alpha(
    num_rules, num_constraints, rand_seed, as_numpy=True
    )
    
    start_time = time.time() 
    actual_sols = solutions_to_dict(
    list(cbb_v2.run(return_objective=True, A=A, t=t)))
    all_times.append(time.time() - start_time)
    
    
    start_time = time.time() 
    actual_sols = solutions_to_dict(
    list(cbb.run(return_objective=True, A=A, t=t)))
    all_times_naive.append(time.time() - start_time)
    
    
    thresh = 72
    prev_m = 1
    start_time = time.time() 
    est = approx_mc2( rules =  rules,
    y=y,
    lmbd = lmbd,
    ub = ub,
    delta =delta,
    eps =  eps)
    all_times_approx_mc2.append(time.time() - start_time)
    


In [ ]:
all_times_naive

In [ ]:
import matplotlib.pyplot as plt 
fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

all_n_rules = [98, 77 , 61, 46]

plt.plot(all_n_rules, all_times_naive, label="CBB-Naive", marker = "o", linewidth=1.5 , color = colors[0],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 
plt.plot(all_n_rules, all_times, label="CBB", marker = "^", linewidth=1.5 , color = colors[1],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 
plt.plot(all_n_rules, all_times_approx_mc2, label="ApproxMC2", marker = "^", linewidth=1.5 , color = colors[2],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 

plt.title("CBB vs CBB-Naive") 
plt.xlabel("# Rules", fontsize = 18)
plt.ylabel("Runtime (s)", fontsize = 18)
plt.legend()
plt.grid()
plt.tight_layout() 
plt.yscale("log")
plt.savefig("plots/comparison_naive_and_v2_compas_approx_mc_075" + ".pdf" )
plt.show()

In [ ]:
X.shape

In [ ]:
all_times_approx_mc2

In [ ]:
all_n_rules = [187, 130, 98, 77, 61, 41]

In [ ]:
all_times_naive

In [ ]:
from bds.meel import approx_mc2

In [ ]:
import matplotlib.pyplot as plt 
fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']


plt.plot(all_n_rules, all_times_naive, label="CBB-Naive", marker = "o", linewidth=1.5 , color = colors[0],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 
plt.plot(all_n_rules, all_times, label="CBB", marker = "^", linewidth=1.5 , color = colors[1],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 
plt.plot(all_n_rules, all_times_approx_mc2rox_mc2rox_mc2rox_mc2, label="ApproxMC2", marker = "^", linewidth=1.5 , color = colors[2],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.7) 


plt.title("CBB vs CBB-Naive") 
plt.xlabel("# Rules", fontsize = 18)
plt.ylabel("Runtime (s)", fontsize = 18)
plt.legend()
plt.grid()
plt.tight_layout() 
plt.yscale("log")
plt.savefig("plots/comparison_naive_and_v2_compas_approx_mc" + ".pdf" )
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']


plt.plot(all_n_rules, all_times_naive, label="CBB-Naive", marker = "o", linewidth=1.5 , color = colors[0],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot(all_n_rules, all_times, label="CBB", marker = "^", linewidth=1.5 , color = colors[1],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.title("CBB vs CBB-Naive") 
plt.xlabel("# Rules", fontsize = 18)
plt.ylabel("Runtime (s)", fontsize = 18)
plt.legend()
plt.grid()
plt.tight_layout() 
plt.savefig("plots/comparison_naive_and_v2_compas" + ".pdf" )
plt.show()

In [ ]:
all_times_naive

In [ ]:
num_rules = len(rules) 

In [ ]:
num_constraints = 5

In [ ]:
rand_seed = 12

In [ ]:
rand_seed = 12
num_constraints = 5
num_rules = len(rules) 
from bds.random_hash import generate_h_and_alpha
A, t = generate_h_and_alpha(
num_rules, num_constraints, rand_seed, as_numpy=True
)

In [ ]:

actual_sols = solutions_to_dict(
list(cbb_v2.run(return_objective=True, A=A, t=t)))


In [ ]:
actual_sols

In [ ]:
all_times_naive = [] 
all_times_V0 = [] 
all_times_V1 = [] 
all_times_V2 = [] 
#
n_rep = 3 
#
lmbd = 0.1 
ub = 0.4
#
for _  in range(n_rep): 
    #
    start_time = time() 
    bbNaive = BranchAndBoundNaive(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbNaive.run(return_objective=True))
    all_times_naive.append(time() - start_time) 
    actualNaive = solutions_to_dict(feasible_solutions)
    #
    print("HLB done")
    #
    start_time = time() 
    bbV0 = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV0.run(return_objective=True))
    all_times_V0.append(time() - start_time)
    actual_V0 = solutions_to_dict(feasible_solutions)
    #
    assert_dict_allclose(actual_V0, actualNaive)
    #
    print("RSSB done")
    #
    start_time = time() 
    bbV1 = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV1.run(return_objective=True))
    all_times_V1.append(time() - start_time)
    actual_V1 = solutions_to_dict(feasible_solutions)
    #
    assert_dict_allclose(actual_V0, actual_V1)
    #
    print("EPLB + RSSB done")
    #
    start_time = time() 
    bbV2 = BranchAndBoundV2(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bbV2.run(return_objective=True))
    all_times_V2.append(time() - start_time)
    actual_V2 = solutions_to_dict(feasible_solutions)
    #
    assert_dict_allclose(actual_V1, actual_V2)
    #
    print("HLB + RSSB done")

In [ ]:
actual_V1

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6.38, 3.7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']


x = [i for i in range(n_rep)]
plt.plot(x, all_times_naive, label="HLB", marker = "o", linewidth=1.5 , color = colors[0],  markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot(x, all_times_V0, label="RSSB", marker = "^", linewidth=1.5 , color = colors[1],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot(x, all_times_V1, label="RSSB + EPLB",  marker = "s", linewidth=1.5 ,  color = colors[2],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.plot(x, all_times_V2, label="RSSB + HLB",  marker = "P", linewidth=1.5 ,  color = colors[3],   markersize = 12 , markeredgewidth=0.5, markeredgecolor= "grey"  , alpha = 0.5) 
plt.title("RSSB VS HLB VS RSSB + EPLB VS RSSB + HLB") 
plt.xlabel("Run", fontsize = 18)
plt.ylabel("Runtime", fontsize = 18)
plt.legend()
plt.grid()
plt.tight_layout() 
plt.savefig("plots/first_comparison_bounds_compas" + ".pdf" )
plt.show()

When there are so few rule equivalent bounds seem to be useless. 

In [ ]:
for k in bbV1.equivalence_classes: 
    print(bbV1.equivalence_classes[k].data_points)
